In [3]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

In [4]:
special_tokens = [
    "<s>", "</s>", "<unk>", "<pad>",                 # Temel tokenler
    "<|system|>", "<|user|>", "<|assistant|>", "<|end|>",  # Chat rolleri
    "<function>", "</function>",                    # Function calling
    "<function_name>", "</function_name>",
    "<arguments>", "</arguments>",
    "<result>", "</result>"
]


In [5]:
tokenizer.train(files=["metin.txt"], vocab_size=30_000, min_frequency=2, special_tokens=special_tokens)

In [6]:
tokenizer.decode(tokenizer.encode("Selam dünya! 🌍🔥1500").ids)

'Selam dünya! 🌍🔥1500'

In [7]:
dir(tokenizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_parameters',
 '_tokenizer',
 'add_special_tokens',
 'add_tokens',
 'decode',
 'decode_batch',
 'decoder',
 'enable_padding',
 'enable_truncation',
 'encode',
 'encode_batch',
 'from_file',
 'get_added_tokens_decoder',
 'get_vocab',
 'get_vocab_size',
 'id_to_token',
 'model',
 'no_padding',
 'no_truncation',
 'normalize',
 'normalizer',
 'num_special_tokens_to_add',
 'padding',
 'post_process',
 'post_processor',
 'pre_tokenizer',
 'save',
 'save_model',
 'to_str',
 'token_to_id',
 'train',
 'train_from_iterator',
 'truncation']

In [9]:
tokenizer.token_to_id("<unk>")

2

In [10]:
dir(tokenizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_parameters',
 '_tokenizer',
 'add_special_tokens',
 'add_tokens',
 'decode',
 'decode_batch',
 'decoder',
 'enable_padding',
 'enable_truncation',
 'encode',
 'encode_batch',
 'from_file',
 'get_added_tokens_decoder',
 'get_vocab',
 'get_vocab_size',
 'id_to_token',
 'model',
 'no_padding',
 'no_truncation',
 'normalize',
 'normalizer',
 'num_special_tokens_to_add',
 'padding',
 'post_process',
 'post_processor',
 'pre_tokenizer',
 'save',
 'save_model',
 'to_str',
 'token_to_id',
 'train',
 'train_from_iterator',
 'truncation']

In [ ]:
import emoji

# Unicode'daki tüm emojileri içeren liste
new_tokens = list(emoji.EMOJI_DATA.keys())

# Tokenizer'a yeni tokenları ekle
tokenizer.add_tokens(new_tokens)

# Güncellenmiş tokenizer'ı kaydet
#tokenizer.save_pretrained("./updated_tokenizer")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached emoji-2.14.1-py3-none-any.whl.metadata (5.7 kB)
Using cached emoji-2.14.1-py3-none-any.whl (590 kB)


5042

In [14]:
tokenizer.decode(tokenizer.encode("Selam dünya! 🌍🔥").ids)

'Selam dünya! 🌍🔥'

In [15]:
print(tokenizer.encode("Selam dünya! # £& + -* /  🌍🔥 我").tokens)

['Sel', 'am', 'ĠdÃ¼nya', '!', 'Ġ', '#', 'Ġ', 'Â', '£', '&', 'Ġ', '+', 'Ġ-', '*', 'Ġ/', 'Ġ', 'Ġ', '🌍', '🔥', 'Ġ', 'æ', 'Ī', 'ĳ']


In [ ]:
tokenizer.get_vocab_size()

21173

In [16]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(
        tokenizer_object=tokenizer,
        unk_token="<unk>",
        pad_token="<pad>",
        bos_token="<s>",
        eos_token="</s>",
    )
tokenizer.save_pretrained("../CrispyTokenizer")

/home/hosman/anaconda3/envs/torchEnv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


('../CrispyTokenizer/tokenizer_config.json',
 '../CrispyTokenizer/special_tokens_map.json',
 '../CrispyTokenizer/tokenizer.json')

In [ ]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_call_one',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_pad',
 '_pad_token_type_id',
 '_processor_class',
 '_save_pretrained',
 '_set_model_sp

In [ ]:
from transformers import PreTrainedTokenizerFast


# ✅ Test cümlesi: özel tokenlar + bilinmeyen sembol + normal cümle
text = "Merhaba 𓂀 dünya"

# 🔄 Tokenizasyon
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)
decoded = tokenizer.decode(ids, skip_special_tokens=False)

# 📊 Çıktılar
print("🔹 Tokenler:")
print(tokens)
print("\n🔹 Token ID'leri:")
print(ids)
print(f"\n🔹 Decode sonucu:\n{decoded}")


🔹 Tokenler:
['M', 'er', 'h', 'aba', 'Ġ', 'ð', 'ĵ', 'Ĥ', 'Ģ', 'ĠdÃ¼nya']

🔹 Token ID'leri:
[60, 276, 87, 1365, 236, 188, 257, 240, 238, 982]

🔹 Decode sonucu:
Merhaba 𓂀 dünya


In [ ]:
print(tokenizer.decode(tokenizer.encode("Selam dünya! # £& + -* /  🌍🔥 我"), skip_special=False ) )

Selam dünya! # £& + -* /  🌍🔥 我


In [ ]:
text = "Merhaba 𓂀 dünya"
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)

print("Tokens:", tokens)
print("Token IDs:", ids)
print("UNK token ID:", tokenizer.unk_token_id)


Tokens: ['M', 'er', 'h', 'aba', 'Ġ', 'ð', 'ĵ', 'Ĥ', 'Ģ', 'ĠdÃ¼nya']
Token IDs: [60, 276, 87, 1365, 236, 188, 257, 240, 238, 982]
UNK token ID: 2


POSİTİON İDS

In [ ]:
import torch


def get_position_ids(input_ids):
    # input_ids: (batch_size, seq_len)
    seq_len = input_ids.size(1)
    return torch.arange(seq_len, device=input_ids.device).unsqueeze(0).expand_as(input_ids)

get_position_ids(torch.tensor(([tokenizer.encode("Selam dünya! 🌍🔥").ids])))

AttributeError: 'list' object has no attribute 'ids'